In [30]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [31]:
# Read the CSV file into a DataFrame
df = pd.read_csv('../EveryTraceOfFampayPeriod/famapp_unique_links.csv')

# Create a new DataFrame to store the results
results = pd.DataFrame(columns=['Link', 'Connect Time', 'Wait Time', 'Total Time', 'Resource Count'])

# Create a session object
s = requests.Session()

In [32]:
# Iterate over the links in the DataFrame
for link in df['Link']:
    try:
        # Record the start time
        start_time = time.time()

        # Make a HEAD request to the link to measure connect time
        response = s.head(link)
        
        # Record the connect time
        connect_time = time.time() - start_time

        # Make a GET request to the link to measure wait time and count resources
        start_time = time.time()
        response = s.get(link)

        # Record the wait time and total time
        wait_time = time.time() - start_time
        total_time = connect_time + wait_time

        # Parse the HTML and count the number of resources
        soup = BeautifulSoup(response.text, 'html.parser')
        resource_count = len(soup.find_all(['link', 'script', 'img']))

        # Add the link and times to the results DataFrame
        results = results.append({'Link': link, 'Connect Time': connect_time, 'Wait Time': wait_time, 'Total Time': total_time, 'Resource Count': resource_count}, ignore_index=True) # type: ignore

        results.to_csv('response_times.csv', index=False)

    except requests.exceptions.RequestException as e:
        print(f"Error: Unable to get {link}. Reason: {e}")

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_25688\1645067342.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_25688\1645067342.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_25688\1645067342.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_25688\1645067342.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_25688\1645067342.py:26: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a

In [33]:
# Read the CSV file into a DataFrame
df = pd.read_csv('response_times.csv')
backup = df.copy()
backup.head()

,Link,Connect Time,Wait Time,Total Time,Resource Count
0,https://famapp.in/,0.156436,0.082100,0.238536,76
1,https://famcard.me/,0.141661,0.060641,0.202302,76
2,https://fampay.in/,0.141551,0.108824,0.250376,76
3,https://fampay.in/about,0.033663,0.096842,0.130505,161
4,https://fampay.in/blog,0.031414,0.421628,0.453043,24


In [34]:
df = df.sort_values(by=['Connect Time'], ascending=False).head(30)

fig1 = go.Figure(data=[go.Bar(x=df['Link'], y=df['Connect Time'])])
fig1.update_layout(title_text='Connect Times')
fig1.write_html("../../fam-report-site/public/Web-Analytics/connect_times.html")
fig1.show()

In [35]:
# take top 30 wait times
df = backup.sort_values(by=['Wait Time'], ascending=False).head(30)

fig2 = go.Figure(data=[go.Bar(x=df['Link'], y=df['Wait Time'])])
fig2.update_layout(title_text='Wait Times')
fig2.write_html("../../fam-report-site/public/Web-Analytics/wait_times.html")
fig2.show()

In [36]:
df = backup.sort_values(by=['Total Time'], ascending=False).head(30)

fig3 = go.Figure(data=[go.Bar(x=df['Link'], y=df['Total Time'])])
fig3.update_layout(title_text='Total Times')
fig3.write_html("../../fam-report-site/public/Web-Analytics/total_times.html")
fig3.show()

In [37]:
df = backup.sort_values(by=['Resource Count'], ascending=False).head(30)

fig4 = go.Figure(data=[go.Bar(x=df['Link'], y=df['Resource Count'])])
fig4.update_layout(title_text='Resource Counts')
fig4.write_html("../../fam-report-site/public/Web-Analytics/resource_counts.html")
fig4.show()

In [38]:
fig5 = go.Figure(data=[go.Scatter(x=backup['Total Time'], y=backup['Resource Count'], mode='markers')])
fig5.update_layout(title_text='Total Time vs. Resource Count', xaxis_title='Total Time', yaxis_title='Resource Count')
fig5.write_html("../../fam-report-site/public/Web-Analytics/total_vs_resource.html")
fig5.show()

In [39]:
fig = go.Figure()

df = backup.sort_values(by=['Total Time'], ascending=False).head(30)

# Connect Times
fig.add_trace(go.Bar(
    x=df['Link'], 
    y=df['Connect Time'], 
    name='Connect Time',
    opacity=0.6
))

# Wait Times
fig.add_trace(go.Bar(
    x=df['Link'], 
    y=df['Wait Time'], 
    name='Wait Time',
    opacity=0.6
))

# Total Times
fig.add_trace(go.Bar(
    x=df['Link'], 
    y=df['Total Time'], 
    name='Total Time',
    opacity=0.6
))

# Resource Counts
fig.add_trace(go.Bar(
    x=df['Link'], 
    y=df['Resource Count'], 
    name='Resource Count',
    opacity=0.6
))

fig.update_layout(
    title_text='Overlapping Times & Resource Counts',
    barmode='overlay'
)

fig.write_html("../../fam-report-site/public/Web-Analytics/overlapped_graph.html")
fig.show()

In [40]:
# make a pairplot of the data
fig = px.scatter_matrix(backup, dimensions=['Connect Time', 'Wait Time', 'Total Time', 'Resource Count'])
fig.write_html("../../fam-report-site/public/Web-Analytics/pairplot.html")
fig.show()

c:\ProgramData\anaconda3\lib\site-packages\plotly\express\_core.py:279: FutureWarning:

iteritems is deprecated and will be removed in a future version. Use .items instead.



In [41]:
stats = df.describe()
print(stats)

       Connect Time  Wait Time  Total Time  Resource Count
count     30.000000  30.000000   30.000000       30.000000
mean       0.456665   1.138099    1.594764       52.833333
std        0.300809   0.252807    0.491391       45.752771
min        0.244776   0.634953    1.173797       28.000000
25%        0.290649   0.950515    1.277627       33.000000
50%        0.438207   1.156960    1.625163       37.000000
75%        0.465073   1.276546    1.711257       37.000000
max        1.936156   1.959636    3.895792      167.000000
